In [12]:
import tensorflow as tf
import time
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dense
from tensorflow.keras import Input
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
import numpy as np                                    
import pandas as pd 
import os
import random
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
warnings.filterwarnings("ignore")

In [40]:
class SimpleNet:
    def __init__(self, input_shape,n_classes, filters=32):
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.filters = filters
        self.model = tf.keras.Sequential()
        
        # self.topology_dict = topology_dict # contains the layers = which function is called when.
    def build_self(self, topology_list):
        self.model.add(Input(self.input_shape))
        for layer in topology_list:
            self.model.add(layer)

        self.model.add(Dense(self.n_classes))
        self.model.compile(optimizer="SGD", loss="MAE", metrics=["MSE"])
        return self.model
        
        # self.model.add()



In [41]:
top_list = [Conv2D(32, (3, 3), activation="relu"),Conv2D(16, (3, 3), activation="relu"), Conv2D(8, (3, 3), activation="relu"), Dense(32, activation="sigmoid"), ]

In [43]:
a = SimpleNet((64,64,3),2)

In [44]:
a.build_self(top_list).summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 60, 60, 16)        4624      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 58, 58, 8)         1160      
_________________________________________________________________
dense_8 (Dense)              (None, 58, 58, 32)        288       
_________________________________________________________________
dense_9 (Dense)              (None, 58, 58, 2)         66        
Total params: 7,034
Trainable params: 7,034
Non-trainable params: 0
_________________________________________________________________
